In [5]:
import torch
import torch.nn as nn
import torch.nn.functional as F 
import numpy as np
import pandas as pd
import torch.optim as opt
import utils
import hyp
from rdkit import Chem
from rdkit.Chem import QED
from environment import Molecule
import replay_buffer
from torch.utils.tensorboard import SummaryWriter
from rdkit.Chem import rdFingerprintGenerator
from rdkit.Chem import Crippen
from rdkit.Chem import Descriptors, QED
from rdkit.Contrib.SA_Score import sascorer

In [6]:
import os
import subprocess
from rdkit import Chem
from rdkit.Chem import AllChem

def smiles_to_pdbqt(smiles: str, output_file: str = "legant.pdbqt"):
    """Конвертирует SMILES в PDBQT через Open Babel."""
    # Создание молекулы из SMILES и добавление водородов
    mol = Chem.MolFromSmiles(smiles)
    mol = Chem.AddHs(mol)
    status = AllChem.EmbedMolecule(mol)
    if status == -1:
        raise RuntimeError("Не удалось сгенерировать 3D-структуру")
    
    # Сохранение во временный файл .mol
    temp_mol = "temp.mol"
    Chem.MolToMolFile(mol, temp_mol)
    if not os.path.exists(temp_mol):
        raise FileNotFoundError("Временный файл не создан")
    
    # Конвертация в PDBQT через Open Babel
    cmd = f"obabel {temp_mol} -O legant.pdbqt"
    result = subprocess.run(
        cmd, 
        shell=True, 
        capture_output=True, 
        text=True
    )

    if result.returncode != 0:
        error_msg = f"Ошибка Open Babel:\n{result.stderr}"
        if "Invalid output format" in result.stderr:
            error_msg += "\nУбедитесь, что Open Babel установлен и добавлен в PATH"
        raise RuntimeError(error_msg)

    if not os.path.exists(output_file):
            raise FileNotFoundError(f"Файл {output_file} не создан")
    
    #os.remove(temp_mol)

def run_vina_docking(protein_pdbqt: str, ligand_pdbqt: str, center: tuple = (27.116, 24.090, 14.936), size: tuple = (10, 10, 10)) -> float:
    """Запускает докинг и возвращает энергию связывания."""
    # Создание конфигурационного файла для Vina
    config = f"""
    receptor = {protein_pdbqt}
    ligand = {ligand_pdbqt}
    out = result.pdbqt
    center_x = {center[0]}
    center_y = {center[1]}
    center_z = {center[2]}
    size_x = {size[0]}
    size_y = {size[1]}
    size_z = {size[2]}
    exhaustiveness = 16
    cpu = 12
    """
    with open("config.txt", "w") as f:
        f.write(config)
    
    # Запуск AutoDock Vina
    with open("log.txt", "w") as log_file:
        result = subprocess.run(
            "vina_1.2.7_win --config config.txt",
            stdout=log_file,
            text=True,
            check=True,
            shell=True
        )
        if result.returncode != 0:
            print("Ошибка Vina:", result.stderr)
            return None
    
    # Извлечение энергии связывания из лога
    with open("log.txt", "r") as f:
        log = f.read()
        affinity_values = []
        for line in log.split("\n"):
            if line.strip().startswith("1"):  # Первая строка с результатами
                parts = line.split()
                if len(parts) >= 2:
                    try:
                        affinity = float(parts[1])
                        affinity_values.append(affinity)
                    except ValueError:
                        continue
    
        # Возвращаем лучшую энергию
        if affinity_values:
            return min(affinity_values)
        else:
            print("Энергии связывания не найдены")
            return None

In [7]:
smiles_to_pdbqt("CC(=O)c1cc(OCCN)c(C)cc1C1=C(C)C(=O)COC=C1")


In [8]:
run_vina_docking("COX-2.pdbqt", "legant.pdbqt")

-6.972

In [ ]:
import warnings
warnings.filterwarnings('ignore')
warnings.filterwarnings('error', category=DeprecationWarning)

In [4]:
class MolDQN(nn.Module):
    def __init__(self, input_length, output_length):
        super(MolDQN, self).__init__()

        self.linear_1 = nn.Linear(input_length, 1024)
        self.linear_2 = nn.Linear(1024, 512)
        self.linear_3 = nn.Linear(512, 128)
        self.linear_4 = nn.Linear(128, 32)
        self.linear_5 = nn.Linear(32, output_length)

        self.activation = nn.ReLU()

    def forward(self, x):
        x = self.activation(self.linear_1(x))
        x = self.activation(self.linear_2(x))
        x = self.activation(self.linear_3(x))
        x = self.activation(self.linear_4(x))
        x = self.linear_5(x)

        return x

In [5]:
from joblib import dump, load
REPLAY_BUFFER_CAPACITY = hyp.replay_buffer_size

irritation_model = load('model_irrit.joblib')
melanin_model = load('model_melanin.joblib')
corneal_model = load('catboost_model_joblib.pkl')

def get_fingerprint(molecule):
    mfpgen = rdFingerprintGenerator.GetMorganGenerator(radius=hyp.fingerprint_radius,fpSize=hyp.fingerprint_length)
    if molecule is None:
        return np.zeros((hyp.fingerprint_length,))
    fingerprint = mfpgen.GetFingerprint(molecule)
    return np.array(fingerprint)

def smiles_to_features(smiles:str):
    mol = Chem.MolFromSmiles(smiles)
    descriptor_names = [name for name, func in Descriptors.descList]
    def calc_descriptors(mol):
        if mol is not None:
            descriptor_values = [func(mol) for name, func in Descriptors.descList]
            return descriptor_values
        else:
            return [None] * len(descriptor_names)
    
    descriptor_values = calc_descriptors(mol)
    
    
    features = pd.Series(dict(zip(descriptor_names, descriptor_values)))
    
    return features

class QEDRewardMolecule(Molecule):
    
    def __init__(self, discount_factor, **kwargs):
        
        super(QEDRewardMolecule, self).__init__(**kwargs)
        self.discount_factor = discount_factor

    def _reward(self):
        
        molecule = Chem.MolFromSmiles(self._state)
        if molecule is None:
            return 0.0
        qed = QED.qed(molecule)
        irrit_proba = irritation_model.predict_proba(np.expand_dims(get_fingerprint(molecule), axis=0))[0, 1]
        melanin_proba = melanin_model.predict_proba(np.expand_dims(get_fingerprint(molecule), axis=0))[0, 1]
        sa_score = sascorer.calculateScore(molecule)
        corneal = corneal_model.predict(smiles_to_features(self._state))
        """try:
            smiles_to_pdbqt(self._state)
            docking = run_vina_docking("COX-2.pdbqt", "legant.pdbqt")
        except:
            docking = 0"""
        
        docking = 0
        return (2*np.sqrt(qed) -( 1/3*sa_score if sa_score > 3.5 else 0.4) * sa_score + 0.1 * melanin_proba + docking + 0.1*(corneal-1.7) - irrit_proba*2) * self.discount_factor ** (self.num_steps_taken)


In [6]:
class Agent(object):
    def __init__(self, input_length, output_length, device):
        self.device = device
        self.dqn, self.target_dqn = (
            MolDQN(input_length, output_length).to(self.device),
            MolDQN(input_length, output_length).to(self.device),
        )
        for p in self.target_dqn.parameters():
            p.requires_grad = False
        self.replay_buffer = replay_buffer.ReplayBuffer(REPLAY_BUFFER_CAPACITY)
        self.optimizer = getattr(opt, hyp.optimizer)(
            self.dqn.parameters(), lr=hyp.learning_rate
        )
        self.times_of_update = 0

    def get_action(self, observations, epsilon_threshold):

        if np.random.uniform() < epsilon_threshold:
            action = np.random.randint(0, observations.shape[0])
        else:
            q_value = self.dqn.forward(observations.to(self.device)).cpu()
            action = torch.argmax(q_value).numpy()

        return action

    def update_params(self, batch_size, gamma, polyak):
        # update target network

        # sample batch of transitions
        states, _, rewards, next_states, dones = self.replay_buffer.sample(batch_size)
        q_t = torch.zeros(batch_size, 1, requires_grad=False)
        v_tp1 = torch.zeros(batch_size, 1, requires_grad=False)
        for i in range(batch_size):
            state = (
                torch.FloatTensor(states[i])
                .reshape(-1, hyp.fingerprint_length + 1)
                .to(self.device)
            )
            q_t[i] = self.dqn(state)

            next_state = (
                torch.FloatTensor(next_states[i])
                .reshape(-1, hyp.fingerprint_length + 1)
                .to(self.device)
            )
            v_tp1[i] = torch.max(self.target_dqn(next_state))

        rewards = torch.FloatTensor(rewards).reshape(q_t.shape).to(self.device)
        q_t = q_t.to(self.device)
        v_tp1 = v_tp1.to(self.device)
        dones = torch.FloatTensor(dones).reshape(q_t.shape).to(self.device)

        # # get q values
        q_tp1_masked = (1 - dones) * v_tp1
        q_t_target = rewards + gamma * q_tp1_masked
        td_error = q_t - q_t_target

        q_loss = torch.where(
            torch.abs(td_error) < 1.0,
            0.5 * td_error * td_error,
            1.0 * (torch.abs(td_error) - 0.5),
        )
        q_loss = q_loss.mean()

        # backpropagate
        self.optimizer.zero_grad()
        q_loss.backward()
        self.optimizer.step()

        if self.times_of_update % 10 == 0:
            with torch.no_grad():
                for p, p_targ in zip(self.dqn.parameters(), self.target_dqn.parameters()):
                    p_targ.data.mul_(polyak)
                    p_targ.data.add_((1 - polyak) * p.data)
            self.times_of_update += 1
        return q_loss
        

In [7]:
from tqdm import tqdm
TENSORBOARD_LOG = True
TB_LOG_PATH = "./runs/dqn/run2"
episodes = 0
iterations = 25000
update_interval = 16
batch_size = 512
num_updates_per_it = 1

if torch.cuda.is_available():
    device = torch.device("cuda")
else:
    device = torch.device("cpu")

environment = QEDRewardMolecule(
    discount_factor=hyp.discount_factor,
    atom_types=set(hyp.atom_types),
    init_mol=hyp.start_molecule,
    allow_removal=hyp.allow_removal,
    allow_no_modification=hyp.allow_no_modification,
    allow_bonds_between_rings=hyp.allow_bonds_between_rings,
    allowed_ring_sizes=set(hyp.allowed_ring_sizes),
    max_steps=hyp.max_steps_per_episode,
)

# DQN Inputs and Outputs:
# input: appended action (fingerprint_length + 1) .
# Output size is (1).

agent = Agent(hyp.fingerprint_length + 1, 1, device)
agent.dqn.load_state_dict(torch.load("best_weights.pt", weights_only=True))


if TENSORBOARD_LOG:
    writer = SummaryWriter(TB_LOG_PATH)

environment.initialize()

eps_threshold = 0.8
batch_losses = []

best_reward = -1000

In [8]:
for it in tqdm(range(iterations)):

    steps_left = hyp.max_steps_per_episode - environment.num_steps_taken

    valid_actions = list(environment.get_valid_actions())

    observations = np.vstack(
        [
            np.append(
                utils.get_fingerprint(
                    act, hyp.fingerprint_length, hyp.fingerprint_radius
                ),
                steps_left,
            )
            for act in valid_actions
        ]
    )

    observations_tensor = torch.Tensor(observations)

    a = agent.get_action(observations_tensor, max(0.1, eps_threshold))

   
    action = valid_actions[a]
    result = environment.step(action)

    action_fingerprint = np.append(
        utils.get_fingerprint(action, hyp.fingerprint_length, hyp.fingerprint_radius),
        steps_left,
    )

    next_state, reward, done = result

    steps_left = hyp.max_steps_per_episode - environment.num_steps_taken

    next_state = utils.get_fingerprint(
        next_state, hyp.fingerprint_length, hyp.fingerprint_radius
    ) 

    action_fingerprints = np.vstack(
        [
            np.append(
                utils.get_fingerprint(
                    act, hyp.fingerprint_length, hyp.fingerprint_radius
                ),
                steps_left,
            )
            for act in environment.get_valid_actions()
        ]
    )  


    agent.replay_buffer.add(
        obs_t=action_fingerprint,  # (fingerprint_length + 1)
        action=0,  # No use
        reward=reward,
        obs_tp1=action_fingerprints,  # (num_actions, fingerprint_length + 1)
        done=float(result.terminated),
    )

    if done:
        final_reward = reward
        
        
        if episodes != 0 and TENSORBOARD_LOG and len(batch_losses) != 0:
            writer.add_scalar("episode_reward", final_reward, episodes)
            writer.add_scalar("episode_loss", np.array(batch_losses).mean(), episodes)
        if episodes != 0 and episodes % 2 == 0 and len(batch_losses) != 0:
            try:
                smiles_to_pdbqt(environment._state)
                docking = run_vina_docking("COX-2.pdbqt", "legant.pdbqt")
            except:
                docking = 0
            print(
                "reward of final molecule at episode {} is {}, qed is {},sa is {}, dock is {} , irrit is {},  corneal is {}, melanin is {} , molecule is {}".format(
                    episodes, final_reward, QED.qed(Chem.MolFromSmiles(environment._state)), sascorer.calculateScore(Chem.MolFromSmiles(environment._state)), docking, 
                    irritation_model.predict_proba(np.expand_dims(get_fingerprint(Chem.MolFromSmiles(environment._state)), axis=0))[0, 1], corneal_model.predict(smiles_to_features(environment._state)) ,
                    melanin_model.predict_proba(np.expand_dims(get_fingerprint(Chem.MolFromSmiles(environment._state)), axis=0))[0, 1] ,  environment._state
                )
            )
            print(
                "mean loss in episode {} is {}".format(
                    episodes, np.array(batch_losses).mean()
                )
            )
        if final_reward > best_reward:
            torch.save(agent.dqn.state_dict(), 'best_weights.pt')
            print(f"ЛУЧШИЕ ВЕСА СОХРАНЕНЫ, награда {final_reward}") 
            best_reward = final_reward
        episodes += 1
        eps_threshold *= 0.998
        batch_losses = []
        environment.initialize()

    if it % update_interval == 0 and agent.replay_buffer.__len__() >= batch_size:
        for update in range(num_updates_per_it):
            loss = agent.update_params(batch_size, hyp.gamma, hyp.polyak)
            loss = loss.item()
            batch_losses.append(loss)

  0%|                                                                               | 33/25000 [00:02<23:52, 17.43it/s]

ЛУЧШИЕ ВЕСА СОХРАНЕНЫ, награда -2.049457660622108


  0%|▏                                                                              | 63/25000 [00:04<26:27, 15.71it/s]

ЛУЧШИЕ ВЕСА СОХРАНЕНЫ, награда -1.3392045257120528


  0%|▎                                                                              | 94/25000 [00:06<26:15, 15.81it/s]

...

 32%|██████████████████████▉                                                 | 7952/25000 [1:24:41<16:59:55,  3.59s/it]

reward of final molecule at episode 264 is 2.482093383973993, qed is 0.5159665100263395,sa is 3.9227031327128854, dock is -4.991 , irrit is 0.004785395973807747,  corneal is 3.4834850695480712, melanin is 0.9719522898240925 , molecule is CNCC(CC(CCOC)ON)OC
mean loss in episode 264 is 0.033611828461289406


 32%|███████████████████████                                                 | 8013/25000 [1:25:32<21:03:12,  4.46s/it]

reward of final molecule at episode 266 is -1.4967349578510443, qed is 0.29610308283063547,sa is 5.389106727530856, dock is 0 , irrit is 0.589425963748347,  corneal is 4.514821403768086, melanin is 0.955733380255363 , molecule is C=NC1(NOOCCC)COC2CC21
mean loss in episode 266 is 0.022205770015716553


 32%|███████████████████████▍                                                 | 8016/25000 [1:25:34<3:01:19,  1.56it/s]

KeyboardInterrupt



In [ ]:
generated_molecules = []
num_molecules_to_generate = 100
agent.dqn.eval()
eps_threshold = 0.03

for it in range(num_molecules_to_generate):
    done = False
    environment.initialize()
    while not done:
        steps_left = hyp.max_steps_per_episode - environment.num_steps_taken
        valid_actions = list(environment.get_valid_actions())
    
        observations = np.vstack(
            [
                np.append(
                    utils.get_fingerprint(
                        act, hyp.fingerprint_length, hyp.fingerprint_radius
                    ),
                    steps_left,
                )
                for act in valid_actions
            ]
        ) 
    
        observations_tensor = torch.Tensor(observations)
        a = agent.get_action(observations_tensor, eps_threshold)
        action = valid_actions[a]
        result = environment.step(action)
    
        action_fingerprint = np.append(
            utils.get_fingerprint(action, hyp.fingerprint_length, hyp.fingerprint_radius),
            steps_left,
        )
    
        next_state, reward, done = result
        steps_left = hyp.max_steps_per_episode - environment.num_steps_taken
    
        next_state = utils.get_fingerprint(
            next_state, hyp.fingerprint_length, hyp.fingerprint_radius
        )  
    
        action_fingerprints = np.vstack(
            [
                np.append(
                    utils.get_fingerprint(
                        act, hyp.fingerprint_length, hyp.fingerprint_radius
                    ),
                    steps_left,
                )
                for act in environment.get_valid_actions()
            ]
        )
        #print(environment._state)
    
    
    generated_molecules.append(environment._state)
    print(generated_molecules[-1])

In [ ]:
generated_molecules

In [ ]:
from rdkit import Chem

valid_smiles = []
for smi in generated_molecules:
    mol = Chem.MolFromSmiles(smi)
    if mol is not None:
        valid_smiles.append(smi)

print(f"Сгенерировано валидных молекул: {len(valid_smiles)}/{len(generated_molecules)}")